# Генеративные состязательные сети

https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py

https://habr.com/post/332000/

![gan](images/gan.png)

GAN-ы состоят из двух частей:

1. Генератор (будем обозначать его $G$), который сэмплит случайные числа из какого-то определенного распределения (например, нормального) и генерируют из них объекты, которые идут на вход второй сети.

2. Дискриминатор $D$, который получает на вход объекты из выборки и созданные генератором, и учится предсказывать вероятность того, что конкретный объект реальный (он выдает скаляр — число от 0 до 1).

GAN-ы нужны для «обучения» распределения очень сложных данных (например, изображений), и применяются очень много где, но не в продакшене, а скорее в ресёрче, потому что их на данный момент очень трудно обучать.

Аналогия: представьте двух людей — один пытается подделать произведения искусства, а второй пытается это распознать. Рано или поздно у нас в результате такой длящейся бесконечно долго игры получаются два очень полезных ресурса — генератор **новых** произведений искусства и распознаватель подделок — оба из которых можно применить где-нибудь ещё (а можно и просто наслаждаться синтетическим искусством).

GAN — это такое прямое прохождение теста Тьюринга. Примерно это и мотивирует все их применения в генеративных моделях. Например, колоризацию «правильно» делать так: обучаем одну сеть, которая раскрашивает, а вторая — определяет качество раскраски. Генерация диалогов или перевода — точно так же, вместо языковой модели — две играющие друг против друга сети.

Все бы прекрасно, но у нас есть проблема — мы не знаем, как мерить качество GAN-ов и вообще всех генеративных моделей. Эта проблема ещё не решена. Ничего умнее мнения независимых сетей-классификаторов («inception score») или субъективных человеческих оценок не придумали. Пока что у нас вообще никаких способов оценить качество модели, кроме того, как посмотреть на данные раз в сколько-то эпох. Это всё заметно тормозит архитектурный поиск.

* Unsupervised NMT
* CycleGAN
* Conditional GAN
* InfoGAN
* Морфинг
* VAE

# Код

В этой тетрадке мы будем научимся писать сети на чуть более низком уровне, потому что стандартный .fit() для наших целей недостаточно гибок.

In [13]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model

import numpy as np
from mnist import X
X = X.reshape(-1, 28, 28)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
latent_dim = 64
batch_size = 64
sample_interval = 50
epochs = 1000

Генератор сделаем полносвязным. Желающие могут написать через deconvolution-ы.

In [ ]:
G = Sequential([
    Dense(256, input_dim=latent_dim),
    LeakyReLU(),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(),
    BatchNormalization(),
    Dense(784),
    LeakyReLU(),
    BatchNormalization(),
    Reshape((28, 28))
])

#G.compile()
G.summary()

In [ ]:
D = Sequential([
    Dense(512, input_dim=784),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dense(64),
    LeakyReLU(),
    Dense(1, activation='sigmoid')
])

D.compile(loss='binary_crossentropy', optimizer='adam')
D.summary()

Теперь сольем их в одну модель, как делали с автоэнкодерами.

In [ ]:
z = Input(shape=(latent_dim,))
img = G(z)
D.trainable = False # тут нам нужно отключить ему градиенты, чтобы при обучении всего GAN-а они не менялись
validity = D(img)
GAN = Model(z, validity)
GAN.compile(loss='binary_crossentropy', optimizer='adam')

Самое содержательное это цикл обучения. Его нужно написать вручную — model.fit() не прокатит.

In [ ]:
# эпохи — это в смысле батчи
for epoch in range(epochs):
    # сначала обучим дискриминатор
    real = X[np.random.randint(0, X.shape[0], batch_size)] # так можно посэмплить батч вручную
    noise = np.random.randn((batch_size, latent_dim))
    fake = G.predict(noise)
    
    d_loss_real = D.train_on_batch(real, np.ones((batch_size, 1)))
    d_loss_fake = D.train_on_batch(fake, np.zeros((batch_size, 1)))
    d_loss = np.add(d_loss_real, d_loss_fake)

    # теперь обучаем генератор
    noise = np.random.randn((batch_size, latent_dim))
    g_loss = GAN.train_on_batch(noise, np.ones((batch_size, 1)))

    print ("%d, D loss: %f, G loss: %f" % (epoch, d_loss[0], g_loss))

    if epoch % sample_interval == 0:
        # выводим то, что в батче
        # дополнительных вычислений это не стоит
        fake = fake.reshape(-1, 28, 28)
        
        fig, axs = plt.subplots(8, 8)
        for i in range(8):
            for j in range(8):
                axs[i,j].imshow(gen_imgs[i*8 + j])
                axs[i,j].axis('off')
        fig.savefig("images/%d.png" % epoch)
        plt.close() # не уверен, что это нужно; я это откуда-то скопировал

# Вассерштейново расстояние и эвристики

Обучение GAN-ов не на игрушечных данных **очень** нестабильно. Придумали множество костылей, чтомы его улучшить:
* Делать лэйблы не 0 и 1, а 0.1 и 0.9 или что-то близкое.
* Выкинуть нафиг эту KL-дивергенцию. Оказывается, она не сходится и может вызывать взрывающие / затухающие градиенты. Вместо неё можно использовать Earth Moving.
* Напрямую делать penalty на слишком большой или слишком маленький градиент.
* На каждую итерацию итератора несколько итераций дискриминатора.